# Assignment 2: Loading Graphs in Networkx

## 0. Import Packages

In [2]:
import networkx as nx
import pandas as pd
import matplotlib.pyplot as plt

## 1. Read the csv file into a data frame

In [3]:
def csv_to_dataframe(path: str) -> pd.DataFrame:
    """ Accepts a path to a CSV file and returns the data as a Pandas DataFrame object.
    
    Parameters
    ----------
        path: String object specifying path to CSV data file.
    
    Returns
    -------
        dataframe: Pandas Dataframe object corresponding to input path.
    """
    
    data = pd.read_csv(path)
    return data

## 2. Create a graph from the data frame

In [4]:
def dataframe_to_digraph(dataframe: pd.DataFrame) -> nx.DiGraph:
    """ Create and return a directed graph generated from input dataframe.
    
    Parameters
    ----------
        dataframe: Pandas Dataframe object.
    
    Returns
    -------
        digraph: Networkx Directed Graph generated from input dataframe.
    """
    
    pass

## 3. Visualize the graph using a shell layout

In [5]:
def graph_shell(digraph: nx.DiGraph) -> None:
    """ Generate visualization of directed graph.
    
    Parameters
    ----------
        digraph: Networkx DiGraph object containing graph to visualize.
    """
    
    pass

## 4. Start analyzing the graph

### 4.1 Who was the employee `that was rated` the most?

In [6]:
def most_rated(digraph: nx.DiGraph) -> str:
    """ Retrieve the name of the employee who recieved the most ratings.
    
    Parameters
    ----------
        digraph: Networkx DiGraph object containing employee rating network.
    
    Returns
    -------
        most_rated: String object containing name of the most rated employee.
    """
    
    pass

### 4.2 Who was the employee `that rated` the most?

In [7]:
def rated_most(digraph: nx.DiGraph) -> str:
    """ Retrieve the name of the employee who supplied the most ratings.
    
    Parameters
    ----------
        digraph: Networkx DiGraph object containing employee rating network.
    
    Returns
    -------
        rated_most: String object containing name of the employee who supplied
            the most ratings.
    """

    pass

### 4.3 Who was the employee with the highest average rate?

In [8]:
def highest_rated(dataframe: pd.DataFrame) -> str:
    """ Retrieve the name of the employee with the highest average rating.
    
    Parameters
    ----------
        dataframe: Pandas Dataframe object.
    
    Returns
    -------
        highest_rated: Name of the employee with the highest average rating.
    """
    
    pass

### 4.4 Who was the employee with the lowest average rate?

In [9]:
def lowest_rated(dataframe: pd.DataFrame) -> str:
    """ Retrieve the name of the employee with the lowest average rating.
    
    Parameters
    ----------
        dataframe: Pandas Dataframe object.
    
    Returns
    -------
        lowest_rated: Name of the employee with the lowest average rating.
    """
    
    pass